<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_naranja@4x.png?raw=true" alt="esquema" />
</div>

# Laboratorio Limpieza de Datos

En este laboratorio usaremos el DataFrame de Netflix completo creado en los primeros laboratorios de Pandas. 

**Instrucciones:**

1. Lee cuidadosamente el enunciado de cada ejercicio.

2. Implementa la solución en la celda de código proporcionada.

3. Documenta todas las funciones creadas durante el ejercicio. 

4. Debes incluir después de cada gráfica la interpretación de las mismas en una celda de markdown. 

In [152]:
#Importación de pandas para trabajar con ello
import pandas as pd
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames




In [153]:
# Carga el dataset de trabajo
df = pd.read_excel("Df_Netflix_completo.xlsx")
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Orginal
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",Documentary,"October 2, 2020",90.0,7.5,English,Si
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,International TV Shows,"After crossing paths at a party, a Cape Town t...",NaN,NaN,NaN,NaN,NaN,NaN
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,NaN,Crime TV Shows,To protect his family from a powerful drug lor...,NaN,NaN,NaN,NaN,NaN,NaN
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,NaN,Docuseries,"Feuds, flirtations and toilet talk go down amo...",NaN,NaN,NaN,NaN,NaN,NaN
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,International TV Shows,In a city of coaching centers known to train I...,NaN,NaN,NaN,NaN,NaN,NaN


## Parte 1: Limpieza y Preparación de Datos

#### Ejercicio 1: Estandarización y limpieza de columnas

En este ejercicio, debes limpiar y estandarizar algunas columnas clave para hacerlas más manejables y consistentes en tus análisis. Específicamente, trabajarás con las columnas `date_added` y `duration` para convertirlas a un formato uniforme y estructurado.

Instrucciones:

1. **Convertir la columna `date_added`**: La columna `date_added` contiene fechas en formato de texto. Debes convertirla a un formato `datetime` que pandas pueda entender y manejar fácilmente.

2. **Limpiar la columna `duration`**: La columna `duration` tiene valores en diferentes formatos como "1 Season", "2 Seasons", "90 min", etc. Tu tarea es extraer el número (ya sea el número de temporadas o la cantidad de minutos) y crear una nueva columna llamada `duration_cleaned` con esos valores estandarizados.


**Resultado Esperado:**
Deberás obtener algo como esto:

| duration   | duration_cleaned |
|------------|-----------------|
| 1 Season   | 1               |
| 90 min     | 90              |
| 2 Seasons  | 2               |
| 45 min     | 45              |
| 3 Seasons  | 3               |

In [154]:
#limipieza date_added
df["date_added"]= pd.to_datetime(df["date_added"])
df["date_added"].info()


<class 'pandas.core.series.Series'>
RangeIndex: 8807 entries, 0 to 8806
Series name: date_added
Non-Null Count  Dtype         
--------------  -----         
8797 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 68.9 KB


In [155]:
#limpieza de duración usaré str.replace
df["duration_clean"] = df["duration"]
df["duration_clean"] = df["duration_clean"].str.replace(" min","")
df["duration_clean"] = df["duration_clean"].str.replace(" Seasons","")
df["duration_clean"] = df["duration_clean"].str.replace(" Season","")
df["duration_clean"] 




0        90
1         2
2       NaN
3       NaN
4         2
       ... 
8802    158
8803      2
8804    NaN
8805    NaN
8806    111
Name: duration_clean, Length: 8807, dtype: object

#### Ejercicio 2: Normalización de la columna `rating`

La columna `rating` tiene diferentes calificaciones como `PG`, `PG-13`, `R`, entre otras. Debes categorizar estas calificaciones en tres grupos:

- **'General Audience'** para calificaciones como `G`, `PG`.

- **'Teens'** para calificaciones como `PG-13`, `TV-14`.

- **'Adults'** para calificaciones como `R`, `TV-MA`.


In [156]:
df["rating"].unique()
#observo que hay valores de duración aquí por lo que los limpiaré
mascara_error_rating = df["rating"].str.contains("min",case=False)
df.loc[mascara_error_rating==True,"duration"]=df.loc[mascara_error_rating==True,"rating"]
df.loc[mascara_error_rating==True,"rating"]=None
df.loc[mascara_error_rating==True,("rating","duration")]

,rating,duration
5541,None,74 min
5794,None,84 min
5813,None,66 min


In [157]:
df["rating"].unique()
#usaré el método .aplly
def rating(rate):
    diccionario = {
        "General Audience": ["G", "PG"],
        "teens": ["PG-13", "TV-14"],
        "Adults": ["R", "TV-MA"]
    }
    for result, valor in diccionario.items():
        if rate in valor:
            return result
    return "desconocido"

df["rating_OK"]= df["rating"].apply(rating)
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Orginal,duration_clean,rating_OK
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",Documentary,"October 2, 2020",90.0,7.5,English,Si,90,teens
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,International TV Shows,"After crossing paths at a party, a Cape Town t...",NaN,NaN,NaN,NaN,NaN,NaN,2,Adults
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,2021-09-24,2021,TV-MA,NaN,Crime TV Shows,To protect his family from a powerful drug lor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adults
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,2021-09-24,2021,TV-MA,NaN,Docuseries,"Feuds, flirtations and toilet talk go down amo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adults
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,TV-MA,2 Seasons,International TV Shows,In a city of coaching centers known to train I...,NaN,NaN,NaN,NaN,NaN,NaN,2,Adults


#### Ejercicio 3: Creación de una columna personalizada basada en el elenco

Vamos a identificar si un actor clave como `Leonardo DiCaprio`, `Tom Hanks`, o `Morgan Freeman` aparece en el elenco.

Usa `apply` y una función lambda para crear una nueva columna llamada `has_famous_actor` que contenga `True` si alguno de estos actores está en la lista de `cast` y `False` en caso contrario.

In [158]:
famous_actors = ["Leonardo DiCaprio", "Tom Hanks", "Morgan Freeman"]

# Uso apply y lambda para crear la nueva columna 'has_famous_actor', lo hago con ayuda de internet.
df['has_famous_actor'] = df['cast'].apply(lambda x: any(actor in x for actor in famous_actors) if isinstance(x, str) else False)


df["has_famous_actor"].unique()

array([False,  True])

#### Ejercicio 4: Creación de una columna personalizada usando lógica condicional

Vamos a crear una columna llamada `is_recent` que identifique si un título fue lanzado en los últimos 5 años.

Crea una función para marcar con `True` si el título es reciente (lanzado en los últimos 5 años) y `False` si no lo es.

In [159]:

def lanz_reciente(año):
    if (año + 5) > 2024:
        return True
    else:
        return False
    
df.head(2)
df["is_recent"]=df["release_year"].apply(lanz_reciente)   
df[["release_year","is_recent"]]


,release_year,is_recent
0,2020,True
1,2021,True
2,2021,True
3,2021,True
4,2021,True
...,...,...
8802,2007,False
8803,2018,False
8804,2009,False
8805,2006,False


#### Ejercicio 5: Clasificación de películas por década

En este ejercicio, tu objetivo es categorizar los años de lanzamiento de las películas o series en décadas. La columna `release_year` contiene el año de lanzamiento y debes crear una nueva columna llamada `decade` que indique la década correspondiente, como "1990s", "2000s", etc.


In [160]:
#veré primero el minimo y el maximo
df["release_year"].describe()

count    8807.000000
mean     2014.180198
std         8.819312
min      1925.000000
25%      2013.000000
50%      2017.000000
75%      2019.000000
max      2021.000000
Name: release_year, dtype: float64

In [161]:
#utilizaré el método cut
df["decade"] = pd.cut(df["release_year"],
                      bins=[1920,1929,1939,1949,1959,1969,1979,1989,1999,2009,2019,2029],
                      labels=["1920s","1930s","1940s","1950s","1960s","1970s","1980s","1990s","2000s","2010s","2020s"]
                      )
df[["release_year","decade"]]

,release_year,decade
0,2020,2020s
1,2021,2020s
2,2021,2020s
3,2021,2020s
4,2021,2020s
...,...,...
8802,2007,2000s
8803,2018,2010s
8804,2009,2000s
8805,2006,2000s


#### Ejercicio 6: Extracción de información

Para practicar la extracción de información:

1. **Extrae el primer actor** de la lista en la columna `cast` y crea una nueva columna llamada `first_actor`.

2. **Extrae el primer nombre del director** y guárdalo en una columna llamada `first_name_director`.


In [162]:
df.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Orginal,duration_clean,rating_OK,has_famous_actor,is_recent,decade
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",Documentary,"October 2, 2020",90.0,7.5,English,Si,90,teens,False,True,2020s
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,International TV Shows,"After crossing paths at a party, a Cape Town t...",NaN,NaN,NaN,NaN,NaN,NaN,2,Adults,False,True,2020s
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,2021-09-24,2021,TV-MA,NaN,Crime TV Shows,To protect his family from a powerful drug lor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adults,False,True,2020s
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,2021-09-24,2021,TV-MA,NaN,Docuseries,"Feuds, flirtations and toilet talk go down amo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adults,False,True,2020s
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,TV-MA,2 Seasons,International TV Shows,In a city of coaching centers known to train I...,NaN,NaN,NaN,NaN,NaN,NaN,2,Adults,False,True,2020s


In [169]:
#creo una función para sacar el primer valor de una lista y le aplico a la columna cast con un apply
cadena = ["hola","que","tal"]
def primer_valor(cadena):
        return cadena[0]

df["cast_OK"]=df["cast"].astype(str).str.split(",")
df["first_actor"]= df["cast_OK"].apply(primer_valor)
df[["cast_OK","first_actor"]]

,cast_OK,first_actor
0,[nan],nan
1,"[Ama Qamata, Khosi Ngema, Gail Mabalane, Th...",Ama Qamata
2,"[Sami Bouajila, Tracy Gotoas, Samuel Jouy, ...",Sami Bouajila
3,[nan],nan
4,"[Mayur More, Jitendra Kumar, Ranjan Raj, Al...",Mayur More
...,...,...
8802,"[Mark Ruffalo, Jake Gyllenhaal, Robert Downe...",Mark Ruffalo
8803,[nan],nan
8804,"[Jesse Eisenberg, Woody Harrelson, Emma Ston...",Jesse Eisenberg
8805,"[Tim Allen, Courteney Cox, Chevy Chase, Kat...",Tim Allen


In [173]:
#para el director hago lo mismo pero usando el espacio para dividir
df["dir_OK"]=df["director"].astype(str).str.split(" ")
df["director_name"]= df["dir_OK"].apply(primer_valor)
df[["director","dir_OK","director_name"]]

,director,dir_OK,director_name
0,Kirsten Johnson,"[Kirsten, Johnson]",Kirsten
1,NaN,[nan],nan
2,Julien Leclercq,"[Julien, Leclercq]",Julien
3,NaN,[nan],nan
4,NaN,[nan],nan
...,...,...,...
8802,David Fincher,"[David, Fincher]",David
8803,NaN,[nan],nan
8804,Ruben Fleischer,"[Ruben, Fleischer]",Ruben
8805,Peter Hewitt,"[Peter, Hewitt]",Peter


#### Ejercicio 7: Limpieza de la columna `cast`

La columna `cast` contiene una lista de actores separados por comas. Tu objetivo es realizar las siguientes tareas:

1. **Reemplaza los valores nulos** en la columna `cast` por "sin información".

2. **Contar el número de actores** en cada entrada y crear una nueva columna llamada `num_cast`.

3. **Normalizar los nombres**: Asegúrate de que los nombres de los actores estén en un formato consistente (por ejemplo, quitar espacios adicionales).



#### Ejercicio 9: Identificación de Directores Recurrentes

En este ejercicio, debes identificar los directores que aparecen más de una vez en el conjunto de datos. Realiza los siguientes pasos:

1. **Reemplaza los valores nulos** en la columna `director` por "sin información".

3. **Cuenta cuántas veces aparece cada director** en la columna creada en el ejercicio 6.

4. **Filtra aquellos directores que aparecen más de una vez** y crea una nueva columna llamada `recurrent_director` donde se indique "Yes" si el director aparece varias veces o "No" en caso contrario.